#Broyden's Method

* Given $x_0$
* Compute $B^{-1}$, where $B = JF(x_0)$
* $x = x0 - B^{-1} F(x_0)$
* for $n \geq 1$
   * Compute $v := B^{-1} (F(x) - F(x_0))$
   * Compute $s = x - x0$
   * Update $B^{-1}_{n+1} = B^{-1}_n + \frac{(s-v)s^T B^{-1}_n}{s^T v}$
   * Compute $x = x - B^{-1}_{n+1} F(x)$

In [10]:
function broyden(F, JF, x, tol)
    k = 1
    Nmax = 1000
    xp = zeros(size(x))
    s = zeros(size(x))
    xp[:] = x[:]
    iB = inv(JF(x))
    x = x - iB * F(x)
    while norm(F(x)) > tol && k < Nmax
        v = iB * (F(x) - F(xp))
        s[:] = x[:] - xp[:]
        iB = iB + (s - v) * s' * iB / dot(s,v)
        xp[:] = x[:]
        x = x - iB * F(x)
        k = k + 1
    end
    return x
end

broyden (generic function with 1 method)

Test problem:
$$
\begin{cases}
3x_1 - \cos(x_2 x_3) - 0.5 &= 0 \\
x_1^2 - 81(x_2 + 0.1)^2 + \sin(x_3) + 1.06 &= 0 \\
e^{-x_1 x_2} + 20x_3 + (10\pi -3 )/3 & = 0
\end{cases},
$$
with the initial guess $x^0 = [0.1,\; 0.1,\; -0.1]^T$.  

The Jacobian matrix $J(x)$ for this system is 
$$
J(x_1, x_2, x_3) =
\begin{bmatrix}
3 & x_3 \sin(x_2 x_3) & x_2 \sin(x_2 x_3) \\
2x_1 & -162(x_2 + 0.1) & \cos(x_3) \\
-x_2 e^{-x_1 x_2} & -x_1 e^{-x_1 x_2} & 20
\end{bmatrix}
$$

In [11]:
function F(x)
    return [ 3*x[1] - cos(x[2]*x[3]) - 1/2; x[1]*x[1] - 81*(x[2]+0.1)^2 + sin(x[3])+1.06; 
        exp(-x[1]*x[2]) + 20*x[3] + (10*pi-3)/3 ]
end

function DF(x)
    mat = [ 3 x[3]*sin(x[2]*x[3]) x[2]*sin(x[2]*x[3]); 
        2*x[1] -162*(x[2]+0.1) cos(x[3]);
        -x[2]*exp(-x[1]*x[2]) -x[1]*exp(-x[1]*x[2]) 20];
    return mat
end

DF (generic function with 1 method)

In [12]:
broyden(F, DF, [1.1;0.1;-0.1], 1e-12)
#[1;1] * [1 1]

3-element Array{Float64,1}:
  0.5        
 -6.29515e-17
 -0.523599   

#Bisection Method

The bisection is used for the root-finding problem, i.e. find $x$ so that $f(x) = 0$, where $f$ is a continuous function.  The main concept behind this method is the  **Intermediate Value Theorem**, which states that if $f(a)f(b) < 0$, then there exists $c \in (a,b)$ such that $f(c) = 0$.

* p = $\frac{a+b}{2}$.
    * if $f(p) = 0$, $p$ is the root
    * if $f(a)f(p) < 0$, that means the root is in $[a,p]$, and hence, set $b = p$
    * if $f(b)f(p) < 0$, that means the root is in $[p,b]$, and hence, set $a = p$
    * repeat

In [13]:
function bisection(f, a, b, tol, Nmax)
    fa = f(a)
    i = 1
    while i < Nmax
        p = a + (b - a)/2
        fp = f(p)
        if (abs(fp)) < tol
            return p
        end
        if fa * fp > 0
            a = p
            fa = fp
        else
            b = p
        end
        i = i + 1
        #println(i, '\t', p, '\t', abs(fp))
    end
end

bisection (generic function with 1 method)

In [14]:
bisection(x->x^2-1, 0, 1.5, 1e-5, 10)

# Gradient Descent Method

In [15]:
function GradDesConst(F, JF, x, tol)
    gradG = x -> 2 * JF(x)' * F(x)
    xp = zeros(size(x))
    k = 1
    Nmax = 2000
    alpha = 1e-4
    while norm(F(x)) > tol && k < Nmax
        xp[:] = x[:]
        x = x - alpha * gradG(x)
        k = k + 1
        println(k, '\t', norm(F(x)))
    end
    return x
end

GradDesConst (generic function with 1 method)

In [16]:
GradDesConst(F, DF, [1.1;0.1;-0.1], 1e-5)

2	7.970027524610165
3	7.370201033093512
4	6.821380318475698
5	6.3193228551653995
6	5.860287396216311
7	5.440914526136627
8	5.0581557350054505
9	4.709226110679503
10	4.391569425783583
11	4.102830260582
12	3.840830499259718
13	3.603548859605807
14	3.38910280528802
15	3.1957325669086134
16	3.021787205757094
17	2.8657127565353027
18	2.7260425114096116
19	2.6013894734059013
20	2.490440926289287
21	2.391954957877377
22	2.3047586556994535
23	2.227747591281618
24	2.1598861425946065
25	2.1002081863872135
26	2.0478177259077954
27	2.0018890971327683
28	1.9616665026822384
29	1.9264627385454274
30	1.895657087364077
31	1.8686924407648486
32	1.845071775540329
33	1.824354143456301
34	1.8061503453946044
35	1.7901184530317966
36	1.77595932156068
37	1.7634122108068135
38	1.7522506040230126
39	1.7422782868405284
40	1.73332572522998
41	1.7252467617775504
42	1.7179156342895252
43	1.7112243094220372
44	1.7050801161698856
45	1.6994036590224024
46	1.6941269877928096
47	1.6891919999964
48	1.6845490517212605
49	

3-element Array{Float64,1}:
  0.516886  
  0.00109294
 -0.523569  

0.6338717421464187
592	0.632737339026082
593	0.6316049629701102
594	0.630474610365349
595	0.6293462776050629
596	0.6282199610889241
597	0.6270956572230008
598	0.6259733624197482
599	0.6248530730979936
600	0.623734785682926
601	0.6226184966060873
602	0.6215042023053582
603	0.6203918992249486
604	0.6192815838153857
605	0.6181732525335025
606	0.6170669018424279
607	0.6159625282115737
608	0.614860128116625
609	0.6137596980395286
610	0.612661234468481
611	0.6115647338979195
612	0.6104701928285087
613	0.6093776077671313
614	0.6082869752268752
615	0.6071982917270243
616	0.6061115537930473
617	0.6050267579565849
618	0.6039439007554422
619	0.602862978733574
620	0.6017839884410765
621	0.6007069264341753
622	0.5996317892752155
623	0.5985585735326489
624	0.5974872757810251
625	0.5964178926009811
626	0.5953504205792279
627	0.5942848563085426
628	0.5932211963877548
629	0.5921594374217387
630	0.5910995760214002
631	0.5900416088036682
632	0.5889855323914822
633	0.5879313434137826
634	0.586879038505498

In [17]:
function GradDesBi(F, JF, x, tol, bisection)
    gradG = x -> 2 * JF(x)' * F(x)
    h = (alpha, x, d) -> dot(gradG(x-alpha*d), d)
    xp = zeros(size(x))
    k = 1
    Nmax = 2000
    alpha = 1e-4
    while norm(F(x)) > tol && k < Nmax
        xp[:] = x[:]
        alpha = bisection(a -> h(a, x, gradG(x)), 0, 1.5, 1e-5, 1000)
        x = x - alpha * gradG(x)
        k = k + 1
        println(k, '\t', norm(F(x)))
    end
    return x
end

GradDesBi (generic function with 1 method)

In [18]:
x0 = GradDesBi(F, DF, [1.1;0.1;-0.1], 1e-4, bisection)

2	

3-element Array{Float64,1}:
  0.498171
 -0.199608
 -0.528827

1.8118948270755966
3	1.7408778349117393
4	1.5987769196164405
5	0.7789083375999117
6	0.6616792761440902
7	0.5642524892963374
8	0.47172340957830017
9	0.39606307395972584
10	0.3436397898201964
11	0.29871136815575
12	0.27635339386022606
13	0.2558005377216454
14	0.23820089648773812
15	0.22183407275613587
16	0.20675275784899672
17	0.1927009303583694
18	0.17962009471450127
19	0.16742857591596483
20	0.15606799256069862
21	0.14547946410605853
22	0.1356108090811642
23	0.12641244034166071
24	0.11783905346842238
25	0.10984782557335954
26	0.10239938188487195
27	0.09545649313041868
28	0.0889851341920356
29	0.08295273049811659
30	0.07732983800761349
31	0.07208848258761262
32	0.067202656118126
33	0.06264816652749253
34	0.05840269858180582
35	0.05444540758223863
36	0.05075623155111957
37	0.04731718071810144
38	0.04411136536585193
39	0.041122672816247964
40	0.03833684819127723
41	0.03574000934537644
42	0.033319047616563825
43	0.03106268224906487
44	0.028959422404435137
45	0.026998896977964842
46	0.02517

In [19]:
broyden(F, DF, x0, 1e-12)

3-element Array{Float64,1}:
  0.498145
 -0.199606
 -0.528826